# Recommendation system model training

### 1) Import relevant libraries
please remove libraries that u r not using...

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from statistics import median
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


### Faster but less thorough version.

In [2]:
csv_file_path = 'clean_train_reco.csv'
df = pd.read_csv(csv_file_path)

# Ignore ConvergenceWarning
warnings.filterwarnings("ignore", category=UserWarning, message=".*ConvergenceWarning.*")

# Separate features and target variables
prediction_columns = ['fixed_deposits', 'loan', 'credit_card_debit_card', 'account']
X = df.drop(columns=prediction_columns)  # First 86 columns (features)
y = df[prediction_columns]

# Normalize the feature columns
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

### Create class for .......
Here we use logistic regression ....

In [ ]:
# Create a class for model evaluation
class ModelEvaluator:
    def __init__(self):
        self.models = {}
        
    def train_models(self, X_train, y_train):
        # Define the logistic regression model
        lr_model = LogisticRegression(max_iter=1000, class_weight='balanced', solver='liblinear')

        # Set up hyperparameters for tuning
        param_dist = {
            'C': [0.001, 0.01, 0.1, 1],  # Reduced number of hyperparameters
            'penalty': ['l1', 'l2'],  # Only two regularization types
        }

        # Use StratifiedKFold for cross-validation
        skf = StratifiedKFold(n_splits=3)  # Reduced number of folds

        # Perform random search with cross-validation
        random_search = RandomizedSearchCV(lr_model, param_dist, n_iter=8, cv=skf, scoring='f1_macro', n_jobs=-1)

        for i in range(y_train.shape[1]):
            print(f"Training model for target column {y_train.columns[i]}")

            # Train the model
            random_search.fit(X_train, y_train.iloc[:, i])
            self.models[y_train.columns[i]] = random_search.best_estimator_
            print(f"Best parameters for {y_train.columns[i]}: {random_search.best_params_}")

        return self.models

    def evaluate_model(self, model, X_test, y_test, column_name):
        y_pred = model.predict(X_test)
        print(y_pred)
        print(f"Confusion Matrix for {column_name}:\n", confusion_matrix(y_test[column_name], y_pred))
        print(f"Classification Report for {column_name}:\n", classification_report(y_test[column_name], y_pred))
        
    def evaluate_all_models(self, X_test, y_test):
        for column_name, model in self.models.items():
            self.evaluate_model(model, X_test, y_test, column_name)

### Model training and evaluation

In [4]:
# Create and train the models
model_evaluator = ModelEvaluator()
models = model_evaluator.train_models(X_train, y_train)

# Evaluate all trained models
model_evaluator.evaluate_all_models(X_test, y_test)

Training model for target column fixed_deposits
Best parameters for fixed_deposits: {'penalty': 'l1', 'C': 0.1}
Training model for target column loan
Best parameters for loan: {'penalty': 'l2', 'C': 1}
Training model for target column credit_card_debit_card
Best parameters for credit_card_debit_card: {'penalty': 'l2', 'C': 1}
Training model for target column account
Best parameters for account: {'penalty': 'l2', 'C': 1}
[0 0 0 ... 1 0 0]
Confusion Matrix for fixed_deposits:
 [[15177  5403]
 [  315  3105]]
Classification Report for fixed_deposits:
               precision    recall  f1-score   support

           0       0.98      0.74      0.84     20580
           1       0.36      0.91      0.52      3420

    accuracy                           0.76     24000
   macro avg       0.67      0.82      0.68     24000
weighted avg       0.89      0.76      0.80     24000

[0 0 0 ... 1 0 0]
Confusion Matrix for loan:
 [[14512  6454]
 [  196  2838]]
Classification Report for loan:
          

### Prediction on new data

Explaination of what the function below is for, and give sample usage

In [ ]:
def predict_proba(new_data):
    probabilities = {}
    for column_name, model in model_evaluator.models.items():
        probabilities[column_name] = model.predict_proba(new_data)[:, 1]  # Probability of class 1
    return probabilities

In [ ]:
# Can give an example of how predict_proba() is used 

### This model takes a while to run, its the more thorough version. Select the code and command + / for mac to undo the # . 

In [ ]:
# # Separate features and target variables
# prediction_columns = ['fixed_deposits', 'loan', 'credit_card_debit_card', 'account']
# X = df.drop(columns=prediction_columns)  # First 86 columns (features)
# y = df[prediction_columns]

# # Normalize the feature columns
# scaler = StandardScaler()
# X_normalized = scaler.fit_transform(X)

# # Should use, stratified train-test split to maintain class distribution, but errors encountered ()
# # one of the classes is has too few rows
# X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# # Create a class for model evaluation
# class ModelEvaluator:
#     def __init__(self):
#         self.models = {}
        
#     def train_models(self, X_train, y_train):
#         # Define the logistic regression model
#         lr_model = LogisticRegression(max_iter=1000, class_weight='balanced')

#         # Set up hyperparameters for tuning
#         param_grid = {
#             'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Inverse of regularization strength
#             'penalty': ['l1', 'l2', 'elasticnet', 'none'],  # Regularization type
#             'solver': ['liblinear', 'saga']  # Solver to use
#         }

#         # Use StratifiedKFold for cross-validation
#         skf = StratifiedKFold(n_splits=5)

#         # Perform grid search with cross-validation
#         grid_search = GridSearchCV(lr_model, param_grid, cv=skf, scoring='f1_macro', n_jobs=-1)
        
#         for i in range(y_train.shape[1]):
#             print(f"Training model for target column {y_train.columns[i]}")
#             # Train the model
#             grid_search.fit(X_train, y_train.iloc[:, i])
#             self.models[y_train.columns[i]] = grid_search.best_estimator_
#             print(f"Best parameters for {y_train.columns[i]}: {grid_search.best_params_}")



#     def evaluate_model(self, model, X_test, y_test, column_name):
#         y_pred = model.predict(X_test)
#         print(f"Confusion Matrix for {column_name}:\n", confusion_matrix(y_test[column_name], y_pred))
#         print(f"Classification Report for {column_name}:\n", classification_report(y_test[column_name], y_pred))
        
#     def evaluate_all_models(self, X_test, y_test):
#         for column_name, model in self.models.items():
#             self.evaluate_model(model, X_test, y_test, column_name)

# # Create and train the models
# model_evaluator = ModelEvaluator()
# model_evaluator.train_models(X_train, y_train)

# # Evaluate all trained models
# model_evaluator.evaluate_all_models(X_test, y_test)

# # After training, you can use the models to make predictions on new data
# def predict_proba(new_data):
#     new_data_normalized = scaler.transform(new_data)
#     probabilities = {}
#     for column_name, model in model_evaluator.models.items():
#         probabilities[column_name] = model.predict_proba(new_data_normalized)[:, 1]  # Probability of class 1
#     return probabilities


